In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

In [2]:
tracks = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')

In [3]:
track_audio_features = pd.merge(audio_features, tracks, on='track_uri')

In [4]:
track_audio_features = track_audio_features.drop(['sp_id','id','pos','album_uri','track_uri','album_name',
                                                  'time_signature','track_name'],axis=1)

In [5]:
track_audio_features['key'] = track_audio_features['key'].astype(int)
track_audio_features['mode'] = track_audio_features['mode'].astype(int)

In [6]:
d = {'artist_name': 'first','acousticness': 'mean', 'danceability': 'mean','energy': 'mean',
     'instrumentalness':'mean','liveness':'mean','mode':'mean','loudness':'mean','speechiness':'mean','tempo':'mean',
     'valence':'mean','duration_ms':'mean','key':'mean'}

In [7]:
track_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2254865 entries, 0 to 2254864
Data columns (total 14 columns):
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                 int64
liveness            float64
loudness            float64
mode                int64
speechiness         float64
tempo               float64
valence             float64
artist_name         object
artist_uri          object
duration_ms         int64
dtypes: float64(9), int64(3), object(2)
memory usage: 258.0+ MB


In [8]:
average_artist = track_audio_features.groupby('artist_uri',as_index=False).aggregate(d).reindex(columns=track_audio_features.columns)

In [9]:
average_artist.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
0,0.074767,0.64000,0.70100,0.000000,1.333333,0.249333,-2.903667,1.00,0.067467,114.888667,0.592000,Thyro & Yumi,spotify:artist:0001ZVMPt41Vwzt1zsmuzp,245319.666667
1,0.244650,0.30000,0.49050,0.000003,3.000000,0.088500,-8.295500,1.00,0.035100,142.154500,0.286500,Jordan Colle,spotify:artist:0001cekkfdEBoMlwVQvpLg,217747.500000
2,0.021174,0.68675,0.65475,0.783750,5.500000,0.089525,-10.043250,0.75,0.052075,126.506750,0.262875,Motion Drive,spotify:artist:0001wHqxbF2YYRQxGdbyER,473104.750000
3,0.461500,0.80600,0.55050,0.000004,8.000000,0.111600,-9.816000,1.00,0.035350,112.485000,0.685500,"""Faron Young, Nat Stuckey""",spotify:artist:0004C5XZIKZyd2RWvP4sOq,155860.000000
4,0.037700,0.44600,0.47600,0.000000,10.000000,0.075200,-7.108000,1.00,0.031600,148.138000,0.070500,The Ruins,spotify:artist:000DnGPNOsxvqb2YEHBePR,261053.000000


#### The Coronas

In [10]:
average_artist[average_artist['artist_uri'].str.contains("spotify:artist:2tppd6KkhK4ULAd217Ecq1")]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
109626,0.183438,0.439107,0.656929,0.004278,5.964286,0.131375,-6.816643,0.678571,0.045946,126.044857,0.432214,The Coronas,spotify:artist:2tppd6KkhK4ULAd217Ecq1,216252.25


#### Deadmau5

In [11]:
average_artist[average_artist['artist_uri'].str.contains("spotify:artist:2CIMQHirSU0MQqyYHq0eOx")]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
83108,0.076071,0.710812,0.672491,0.601811,5.854651,0.214057,-8.310573,0.526163,0.096417,127.901227,0.33042,deadmau5,spotify:artist:2CIMQHirSU0MQqyYHq0eOx,394589.947674


#### Andrea Bocelli

In [12]:
average_artist[average_artist['artist_name'] =="Andrea Bocelli"]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
122013,0.796163,0.35149,0.416778,0.017042,5.014151,0.238175,-9.085057,0.79717,0.056493,109.542642,0.244869,Andrea Bocelli,spotify:artist:3EA9hVIzKfFiQI0Kikz2wo,239480.627358


### Top 5 Artists with highest Tempo
<br> Top artists are DJ's and Fast Country Fiddle Players

In [13]:
average_artist.sort_values(by='tempo', ascending=False).head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
105461,0.000854,0.686,0.493,0.000098,11.0,0.0821,-8.758,1.0,0.0771,249.915,0.818,Mark Graham,spotify:artist:2n2ZWyoCn0jr3I5rhDoFAw,410517.0
135192,0.003550,0.515,0.704,0.867000,11.0,0.0902,-7.549,0.0,0.1770,249.703,0.534,Brisker & Magitman,spotify:artist:3Zkbq73VKGYrRyJtaxC4Jg,425533.0
35341,0.000008,0.374,0.561,0.810000,10.0,0.1270,-11.670,0.0,0.0893,248.733,0.108,Lo:rise,spotify:artist:0vjuYEqk1A6kBzNVpe7aHL,325160.0
61187,0.000022,0.652,0.803,0.927000,11.0,0.0262,-9.985,0.0,0.1830,248.028,0.539,Third Deck,spotify:artist:1c2N9gT2196mWhj4zPjSIq,441290.0
134190,0.000414,0.653,0.610,0.933000,9.0,0.0798,-12.165,1.0,0.1220,247.996,0.198,Mihai Popviciu,spotify:artist:3Y6oMIfZJChZPDTYzFnZBH,420498.0


### Top 5 Artists with highest Instrumentalness
<br> All these artists contain songs with white noise relaxation etc.

In [14]:
average_artist.sort_values(by='instrumentalness', ascending=False).head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
212168,0.99600,0.775,0.891,1.000,0.0,0.223,-10.082,1.0,0.0693,211.641,0.13500,Care Free Sounds and Music,spotify:artist:5asbN4UHCWTgtGEpd2SkHP,30006.0
55932,0.94600,0.160,0.278,1.000,1.0,0.829,-33.681,1.0,0.0523,86.417,0.00001,Traveler's Paradise Nature Sounds,spotify:artist:1TUXJOXGX1udUEBpjpgUmO,2960065.0
220304,0.11600,0.340,0.273,1.000,2.0,0.297,-45.549,1.0,0.0544,133.526,0.03800,Stock Music,spotify:artist:5oDusj8QExWIPXC04Xq7Cd,302000.0
17293,0.00119,0.000,0.342,1.000,9.0,0.392,-14.959,0.0,0.0000,0.000,0.00000,The White Noise Zen & Meditation Sound Lab,spotify:artist:0SS2yPrbIYS1j5r6nJOiWD,148000.0
78850,0.90700,0.267,0.140,0.999,9.0,0.152,-9.357,1.0,0.0382,72.349,0.03060,Dr. Bijay Saabye,spotify:artist:25JxkFzzcKmVxfmdyfwozl,599757.0


### Top 5 Artists with highest Loudness
<br> All these artists are hard rock.

In [15]:
average_artist.sort_values(by='loudness', ascending=False).head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
109128,0.217000,0.3910,0.999,0.139000,2.0,0.484,2.8900,1.0,0.19200,121.509,0.1740,Autotrash,spotify:artist:2szesSbRodgjv4MmSNmsq3,102840.0
195543,0.013200,0.5380,0.832,0.000003,10.0,0.485,2.2130,1.0,0.03830,125.605,0.7970,Doozies,spotify:artist:59kKAjhX9biH9kXj9ngA5a,130000.0
139944,0.000050,0.2220,0.931,0.974000,0.0,0.263,2.1210,1.0,0.05510,73.875,0.1460,Rituals,spotify:artist:3hhajQnpU7xqIBbg49BRQF,118000.0
157035,0.286750,0.3240,0.990,0.003979,7.0,0.244,1.7870,0.5,0.21350,106.366,0.1610,Deviant Angel,spotify:artist:49aq4Bj9g72pgwKaqJ5IH5,256391.5
128675,0.018795,0.1927,0.958,0.006130,9.0,0.440,1.7385,0.5,0.06545,89.487,0.5305,Dope Sweater,spotify:artist:3P3nIUZenxSaXJoVRDtlNb,161763.0


### Top 5 Artists with highest Acousticness
<br> Artist's in this example like: Katherine & Christeen Shipp contain little no instruments and just singing

In [16]:
average_artist.sort_values(by='acousticness', ascending=False).head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
96574,0.996,0.315,0.02150,0.97200,3.0,0.131,-21.510,1.0,0.0485,130.667,0.380,Guelfo Nalli,spotify:artist:2YJbiVNXO1TLizDhRvFlUY,151536.0
177699,0.996,0.507,0.46500,0.18000,11.0,0.311,-10.983,1.0,0.0742,92.432,0.973,Wonder,spotify:artist:4gsp1R5vRaGc74H0oMUiQW,170080.0
135169,0.996,0.394,0.00698,0.90500,9.0,0.104,-20.401,0.0,0.0373,100.025,0.241,Mirja Kempe,spotify:artist:3ZiCWpMOQF6S8zoXrX6ulu,234835.0
116458,0.996,0.738,0.29100,0.86800,7.0,0.243,-24.600,1.0,0.0596,93.185,0.907,James Trujillo,spotify:artist:34yPLb2PKtoUpDKGh9XMyI,94091.0
131311,0.996,0.600,0.19500,0.00193,5.0,0.153,-14.494,1.0,0.7140,215.163,0.220,Katherine & Christeen Shipp,spotify:artist:3TDVoC9zxmIDaQ5Q6rfhJE,40588.0


### Top 5 Artists with highest Liveness
<br> Liveness detects audiences in the audio features
<br> These artists songs all contain audiences and live performances

In [17]:
average_artist.sort_values(by='liveness', ascending=False).head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
143134,0.6730,0.512,0.389,0.000145,10.0,1.000,-16.590,1.0,0.4110,114.880,0.295,"Ruthie Henshall, Julia McKenzie, Bernadette Pe...",spotify:artist:3my3g9ePrp6OppeBa0h8uS,258426.0
116506,0.0285,0.441,0.851,0.002430,9.0,0.997,-4.848,0.0,0.0547,129.994,0.356,Xose Manuel Budiño,spotify:artist:3543vcRAqp201HZK22d0DZ,262693.0
30192,0.3440,0.528,0.831,0.000011,0.0,0.996,-8.048,1.0,0.1090,143.688,0.808,Los Players,spotify:artist:0nNbJRBJ8Cp4Dqv6l1jmuh,234613.0
81338,0.0101,0.551,0.920,0.926000,9.0,0.994,-10.974,1.0,0.0731,173.877,0.113,Charly Lownoise,spotify:artist:29NIIAIq5CRkVuzkggpS0W,331572.0
129527,0.4210,0.481,0.927,0.000000,7.0,0.994,-4.070,1.0,0.1730,108.124,0.443,Pastor Woodrow Hayden and Shiloh,spotify:artist:3QJQaloh81ckn2qr2n7x8N,380373.0


### Top 5 Artists with highest Valence
<br> Valence detects happieness in songs
<br> These artists songs all contain happy and cheerful music

In [18]:
average_artist.sort_values(by='valence', ascending=False).head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_name,artist_uri,duration_ms
284406,0.994,0.753,0.524,0.96000,7.0,0.1100,-15.906,1.0,0.0856,119.991,1.000,Saint Patrick's,spotify:artist:7i2L7GnbtKWC9at38X7fN5,151626.0
84877,0.215,0.777,0.472,0.00001,5.0,0.2770,-7.812,1.0,0.0877,132.774,1.000,Video Rola,spotify:artist:2FD9bsEyjTYNn4pCoaN3Cv,130466.0
17569,0.776,0.695,0.940,0.86600,5.0,0.1000,-7.514,1.0,0.0481,134.988,0.999,tonight.burns.red,spotify:artist:0Sw09Y8dRpdZXedzrjcK95,330666.0
154592,0.697,0.859,0.800,0.84100,3.0,0.0492,-9.188,1.0,0.0524,128.423,0.998,Pustertaler Alpentrio,spotify:artist:45dKKHH0mCBoOMaq5sYOOL,149533.0
254821,0.984,0.837,0.403,0.00450,9.0,0.1000,-10.780,0.0,0.3580,99.508,0.998,Meow Mix Cat Food,spotify:artist:6icBygjIpMcw6gsDqsRd6P,32626.0
